# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

# проверим названия столбцов
data.columns
# названия столбцов оформлены корректно - змеиный регистр, нет лишних символов

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [2]:
# посмотрим на общую информацию о таблице 
data.info()

# 1. Мы имеем датафрейм размерностью 12 х 21525. В нем содержатся данные типа int64, float64 и object (str). 
# 2. Имеются пропуски в столбцах days_employed и total_income, в обоих столбцах пропущено 2174 значения. 
# Проверим, в одних и тех же или в разных строках пропущены значения

data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


да, общий трудовой стаж и ежемесячный доход содержат пропуски в одних и тех же строках. 
Возможно, при состоавлении датафрейма клиенты банка отказались называть свой доход и стаж 
по тем или иным причинам. Т.е. вероятно это пропуски типа MAR.
Хотя как, в таком случае, как им дали кредит? Как вариант, можно связаться с сотрудниками банка 
и уточнить, возможна ли такая ситуация. 
По умолчанию заполним эти значения средним значением по столбцу (mean) в следующем шаге

In [3]:
# посмотрим первые несколько строк и сводную статистику
display(data.head(10))
display(data.describe())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


есть проблемы  с данными: 

    столбец children содержит отрицательные значения и значение 20
    столбец days_employed содержит отрицательные значения и сильновеликие значения 
    (трудовой стаж 401755 / 365 -    это 1100 лет)
    столбец dob_years таже содержит некорректные значения - возраст 0 не может быть
    столбец education с типом данных object не приведен к общему регистру, поправим в следующем шаге
    в остальных столбцах с первого взгляда некорректных значений не наблюдается
    разберемся с каждым столбцом отдельно

In [4]:
# children - посмотрим сколько может быть детей у клиентов банка

display(data.groupby(by='children')['children'].count().sort_values(ascending=False))

# В среднем количество детей у клиентов банка колеблется от 0 до 5, но вдруг появляется цифра 20, 
# что похоже на "вброс"
# Вероятно, это ошибка связана с формированием или копированием датафрейма - у единицы 
# появилось отрицательное значение, а 2 изменилась на 20. 
# Тем более что в следующем столбце days_employed также существуют отрицательные значения 
# в достаточно большом количестве.
# посмотрим остальные значения строк, где у клиентов банка -1 и 20 детей
display(data[data['children'] == -1].head(10))
display(data[data['children'] == 20].head(10))

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
3396,20,NaN,56,высшее,0,женат / замужем,0,F,компаньон,0,NaN,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


в этих строках сохранена важная информация о семейном положении и задолженности по возврату кредита, 
не будем их drop-пать (хотя их не много, можно и убрать их)
уберем знак у -1, а 20 изменим на 2 в следующем шаге

In [5]:
# days_employed - посмотрим какие в нем хранятся данные через сводную таблицу
data['days_employed'].describe()
# содержит как отрицательны значения, так и сильно преувеличенные значения. 

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [6]:
# dob_years, посмотрим какие возраста у клиентов банка и их частоты
data.groupby(by='dob_years')['dob_years'].count().sort_values(ascending=False)
# Чаще кредиты берут от 25 до 60 лет, есть некорректные значения 0 у 101 клиента. 
# Это может быть человеческая или техническая ошибка.

dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [7]:
# education, посмотрим все уникальные значения
data['education'].sort_values().unique()
# нужно привесити их к общему (нижнему) регистру в следующем шаге

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [8]:
# famili_status
# определим какие уникальные значения есть в столбце famili_status
data['family_status'].sort_values().unique()
# всего 5 значений, все записи корректные

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [9]:
# gender
# определим распределение клиентов по полу
display(data.groupby(by='gender')['gender'].count().sort_values(ascending=False))

# одно из значений равно XNA, посмотрим эту строку. Одна из страниц поискового запроса яндекса говорит о том, что сущесвуют 54 пола, но не один из них нельзя подогнать под эту аббривеатуру или иное сокращение

# посмотрим на строку целиком, есть ли там еще "битые ячейки"
data[data['gender'] == 'XNA']

# 10701 строка, в которой в столбце gender некорректное значение XNA

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [10]:
# income_type
# посмотрим уникальные значения
data['income_type'].unique()
# всё ровно

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [11]:
# debt
# изучим уникальные значения и их количество
data.groupby(by='debt')['debt'].count().sort_values(ascending=False)
# 1741 клиент имел задолженность по возврату кредитов

debt
0    19784
1     1741
Name: debt, dtype: int64

In [12]:
# total_income
display(data['total_income'].isna().sum())
# да, общий трудовой стаж и ежемесячный доход содержат пропуски в одних и тех же строках. 
# Возможно, при состоавлении датафрейма клиенты банка отказались называть свой доход и 
# стаж по тем или иным причинам. Т.е. вероятно это пропуски типа MAR.
# Заполним эти пропуски средними значениями в слудующем Шаге

# оценим разброс значений
display(data['total_income'].describe())
# разброс значений приемлемый

2174

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [13]:
# purpose
# уникальные значения
data['purpose'].unique()

# в этом столбце можно использовать лемматизацию, чтобы можно было отсортировать данные. 
# в основном цели кредита можно разделить на:
#     приобретение или ремонт автомобиля
#     праздники (свадьба)
#     операции над недвижимостью
#     образование

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

В таблице есть проблемы с данными, которые необходимо исправить перед дальнейшей работы с ней. Из выявленных проблем можно отметить:

а) пропуски значений (в total_income и days_employed). Вероятно это пропуски типа MAR, т.е. их изменение (не учитывание) не должно значимо повлиять на решение задачи. Решено заменить пропущенные значения на нулевые)

б) некорректные (отрицательные или сильно увеличенные) значения в столбцах children, days_employed, исправить знак и убрать 0. Технические ошибки (изменение знака), или человеческий фактор (например, не там поставили запятую).

в) в столбце dob_years есть некорректные значения - 0 в 101 ячейках, заменим их средним значением по столбцу (количественный признак). Тут скорее всего либо техническая ошибка.

г) в стоблце education нужно привести все значения к одному регистру (избавиться от дубикатов)

д) в столбце gender нужно исправить 1 значение

е) в столбце purpose разрозенные занчения можно привести в порядок посредством лемматизации

## Шаг 2. Предобработка данных

### Обработка пропусков

In [14]:
# пропуски значений в total_income и days_employed заменим на среднее значение по столбцу
mean_total_income = data['total_income'].mean().astype(int)

data = data.fillna(value=mean_total_income)

display(data.info())
display(data.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Пропуски обработаны, заполнены средним значением

### Замена типа данных

In [15]:
# Уберем из столбца children отрицательные знаки - возведем каждое значение в модуль:
data['children'] = data['children'].apply(lambda x: abs(x))
data.loc[data['children'] == 20, 'children'] = 2


# data = data.loc[data['children'] < 6] # или же уберем их из датасета
# проверка
display(data.groupby(by='children')['children'].count().sort_values(ascending=False))

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [16]:
# days_employed
# заменим пропуски нулями
data['days_employed'] = data['days_employed'].fillna(value=0)

# Возьмем модуль значений days_employed, для того чтобы избавиться от отрицательных значений
data['days_employed'] = data['days_employed'].apply(lambda x: abs(x))

# также исправим формат с float64 на int64. Испльзовал метод astype, потому что он для этого и создан
data['days_employed'] = data['days_employed'].astype(int)

# проверка
data['days_employed'].describe()
# data.head(20)

count     21525.000000
mean      77065.397677
std      135257.150771
min          24.000000
25%        1025.000000
50%        2609.000000
75%      167422.000000
max      401755.000000
Name: days_employed, dtype: float64

In [17]:
# в столбце dob_years исправим значение 0 на среднее значение по столбцу
# это количественный признак, нулевые значения ошибочны
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean()

# изменить тип данных на int64 (для экономии памяти и ускорения работы)
data['dob_years'] = data['dob_years'].astype(int)

# проверка
data['dob_years'].describe()

count    21525.000000
mean        43.495145
std         12.218213
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [18]:
# gender
# заменим значение в строке 10701 на F
data.loc[10701, 'gender'] = 'F'

# проверка
# data['gender'].unique()
# data.head(20)



In [19]:
# total_income - по аналогии с days_employed
data['total_income'] = data['total_income'].fillna(value=0)
data['total_income'] = data['total_income'].apply(lambda x: abs(x))
data['total_income'] = data['total_income'].astype(int)

# проверка
# data.describe()
data.head(20)
# data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Обработал таблицу, все значения float изменил на int - для более понятного представления таблицы, экономии ресурсов, в том числе временных. 

### Обработка дубликатов

In [20]:
# education - приведем все к нижнему ригистру
data['education'] = data['education'].str.lower()

# проверка
data['education'].unique()

# проверим таблицу на дубликаты строк
# data.duplicated().sum()
# 71 дубликат, посмотрим что это за дубликаты
# data[data.duplicated() == True].describe()
# data.head()


array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

Заменены дубликаты с разными регистрами.
Дубликаты по всей таблице не абсолютные, отличаются семейным статусом и количеством детей, поэтому оставим их.

### Лемматизация

In [21]:
# подключим библиотеку pymystem и collections
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

# создадим список всех текстовых значений столбца purpose и соединим их пробелом
text = ' '.join(data['purpose'])

# лемматизируем текст и выведем на экран посчитав частоты слов
lemmans = m.lemmatize(text)

print(Counter(lemmans))


Counter({' ': 55201, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'подержать': 858, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'приобретение': 462, 'профильный': 436, 'подержанный': 110, '\n': 1})


**Вывод**

т.е. кредиты беруться для следующих задач:

1) на покупку/приобретение/строительство жилья/недвижимости

2) на покупку/ремонт автомобиля

3) на образование

4) на свадьбу

### Категоризация данных

In [22]:
# чтобы ответить на вопрос "влияет ли количество детей на факт погашения кредита в срок", нужно 
# определить какая часть из каждой категории людей имела задолжности. Для этого 
# нужно количество задолжностей категории разделить на количество каждой категории

# создадим сгруппированную таблицу по количеству детей
data_group_children = data.groupby(by='children').count()

# возьмем series из сгруппированного датафрейма с суммой просрочек по уплате
total_debt_cat = data.groupby(by='children')['debt'].sum()
# и series с общим количеством клиентов 
count_debt_cat = data.groupby(by='children')['debt'].count()

# посмотрим на них
# display(count_debt_cat)
# display(total_debt_cat)

# добавим к сгруппированой сохраненной таблице эти колонки и новую, которая получится 
# при делении количество "задолженников" на общее количество

data_group_children['children_to_debt'] = 100 * total_debt_cat / count_debt_cat
data_group_children['total_debt_cat'] = total_debt_cat
data_group_children['count_debt_cat'] = count_debt_cat

# выведем сводную таблицу, те столбцы, которые нас интересуют
data_group_children.loc[:, ['total_debt_cat', 'count_debt_cat', 'children_to_debt']].sort_values(by='children_to_debt')


,total_debt_cat,count_debt_cat,children_to_debt
children,,,
5,0,9,0.000000
0,1063,14149,7.512898
3,27,330,8.181818
1,445,4865,9.146968
2,202,2131,9.479118
4,4,41,9.756098


In [23]:
# аналогичено сделаем с семейным положением. Категории у нас уже имеются (family_status)
# Сгруппируем по ним таблицу и сохраним в другую:

data_group_by_famili_status = data.groupby(by='family_status').count()
total_debt_famili_status = data.groupby(by='family_status')['debt'].sum()
count_debt_famili_status = data.groupby(by='family_status')['debt'].count()

data_group_by_famili_status['debt_family'] = 100 * total_debt_famili_status / count_debt_famili_status
data_group_by_famili_status['total_debt_famili_status'] = total_debt_famili_status
data_group_by_famili_status['count_debt_famili_status'] = count_debt_famili_status

data_group_by_famili_status.loc[:, ['total_debt_famili_status','count_debt_famili_status','debt_family']].sort_values(by='debt_family')

,total_debt_famili_status,count_debt_famili_status,debt_family
family_status,,,
вдовец / вдова,63,960,6.562500
в разводе,85,1195,7.112971
женат / замужем,931,12380,7.520194
гражданский брак,388,4177,9.288963
Не женат / не замужем,274,2813,9.740491


**Вывод**

Дети
Сам факт наличия детей снижает вероятность оплатить кредит в срок, при этом у заемщиков с 3-мя детьми ситуация по своевременному возврату чуть лучше (вероятно связано со льготами), но в целом отмечается тенденция что чем больше детей, тем меньше шансов оплаты кредита в срок.

Семейное положение
    Оказывается, семейное положение также влияет на вероятность оплаты кредита в срок:
    - самые высокие шансы оплатить кредит в срок у вдовца/вдовы
    - далее идут люди в разводе
    - после них женатые/замужние
    - предпоследние те, ко в гражданском браке
    - меньше всего шансов у не женатых/не замужних

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сам факт наличия детей снижает вероятность оплатить кредит в срок, при этом у заемщиков с 3-мя детьми ситуация по своевременному возврату чуть лучше (вероятно связано со льготами), но в целом отмечается тенденция что чем больше детей, тем меньше шансов оплаты кредита в срок.

**Вывод**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?


Семейное положение

    Оказывается, семейное положение также влияет на вероятность оплаты кредита в срок:
    - самые высокие шансы оплатить кредит в срок у вдовца/вдовы
    - далее идут люди в разводе
    - после них женатые/замужние
    - предпоследние те, ко в гражданском браке
    - меньше всего шансов у не женатых/не замужних

**Вывод**

Дети
Сам факт наличия детей снижает вероятность оплатить кредит в срок, при этом у заемщиков с 3-мя детьми ситуация по своевременному возврату чуть лучше (вероятно связано со льготами), но в целом отмечается тенденция что чем больше детей, тем меньше шансов оплаты кредита в срок.

Семейное положение
    Оказывается, семейное положение также влияет на вероятность оплаты кредита в срок:
    - самые высокие шансы оплатить кредит в срок у вдовца/вдовы
    - далее идут люди в разводе
    - после них женатые/замужние
    - предпоследние те, ко в гражданском браке
    - меньше всего шансов у не женатых/не замужних

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
# посмотрим диапазон изменения доходов в месяц 
data['total_income'].max()
data['total_income'].min()

# # max - 2 265 064 000
# # min - 20 667

# # Выделим категории:
# #     1. < 50 тр
# #     2. 50-100 тр
# #     3. 100 - 300 тр
# #     4. > 300

# создадим функцию, которая будет определять к какой категории относится клиент по уровню дохода

def incom_lvl(total_income):
    if total_income < 50*10**3: 
        return '< 50'
    elif 50*10**3 <= total_income < 100*10**3: 
        return '50-100'
    elif 100*10**3 <= total_income <= 300*10**3: 
        return '100-300'
    else: 
        return '> 300'

# создадим новый столбец в таблице и заполним его категориальными данныхми с помощью функции
data['income_lvl'] = data['total_income'].apply(incom_lvl)

# data.head(10)

# создадим новый датафрейм где будут сгруппированы значения по новым категориям:

data_income = data.groupby('income_lvl').count()

# у основной массы клиентов доход от 100 до 300 тр/месяц. Посмотрим кто из них
# лучше платит по счетам

count_income = data.groupby(by='income_lvl')['debt'].count()
total_income = data.groupby(by='income_lvl')['debt'].sum()

# display(total_income)
# display(count_income)

data_income['income_debt'] = 100 * total_income / count_income


data_income.loc[:, ['income_debt']].sort_values(by='income_debt')


,income_debt
income_lvl,
< 50,6.182796
> 300,7.152497
50-100,8.090931
100-300,8.222080


**Вывод**

Лучше всего кредит оплачивают люди с небольшим заработком, наверное потому что им есть что терять.
Также люди с прибылью более 300 тр оплачивают кредит в срок.
А вот остальные показали наименьшие результаты.

- Как разные цели кредита влияют на его возврат в срок?

В пункте 2.4. мы получили такой вывод 

Counter({' ': 55201, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'подержать': 858, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'приобретение': 462, 'профильный': 436, 'подержанный': 110, '\n': 1}) 

сделаем следующий вывод: люди брали кредит для следующих потребностей:

1. связанные с недвижимостью: [недвижимость, жилье]

2. Автомобиль: [автомобиль]

3. Образование: [образование]

4. Свадьба: [свадьба]

Т.о. у нас 4 категории

Нужно написать функцию, которая отнесет цель кредита к одной из 4х категорий:


In [25]:
# загрузим инструменты если вдруг еще не загружены
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

# print(m.lemmatize('покупка жилья для семьи'))

def purpose_func(text):
    lemmans = m.lemmatize(text)
    if 'недвижимость' in lemmans:
        return 'недвижимость'
    elif 'жилье' in lemmans:
        return 'недвижимость'
    elif 'автомобиль' in lemmans:
        return 'автомобиль'
    elif 'образование' in lemmans:
        return 'образование'
    elif 'свадьба' in lemmans:
        return 'свадьба'

# категоризуем данные
data['purpose_func'] = data['purpose'].apply(lambda x: purpose_func(x))

# проверим все ли откатегоризованы
data['purpose_func'].isna().sum()

# и как это выглядит
# data.head(20)

# сгруппируем по новой колонке, создав новую таблицу
data_purpose = data.groupby(by='purpose_func').count()

total_debt_purpose = data.groupby(by='purpose_func')['debt'].sum()
count_debt_purpose = data.groupby(by='purpose_func')['debt'].count()

data_purpose['purpose_debt'] = 100 * total_debt_purpose / count_debt_purpose



data_purpose.loc[:, ['purpose_debt']].sort_values(by='purpose_debt')

,purpose_debt
purpose_func,
недвижимость,7.214022
свадьба,7.921635
образование,9.199403
автомобиль,9.339513


**Вывод**

Клиенты, берущие кредит на операции с недвижимостью, обладают самым высоким шансом уложиться с оплатой в срок

За ними идут клиенты, которым нужны деньги на проведение свадьбы

После них люди, с кредитом на образование

И хуже всего вероятность уложиться в срок у клиентов, чей кредит связан с операциями с авто

In [27]:
data.groupby(by='purpose_func')['debt'].mean()

purpose_func
автомобиль      0.093395
недвижимость    0.072140
образование     0.091994
свадьба         0.079216
Name: debt, dtype: float64

## Шаг 4. Общий вывод

Т.о. в результате анализа датасета от банка мы определили что наличие детей снижает вероятность уложиться по уплате в срок, женатые/замужние клиенты хуже справляются с оплатой, а вдовцы/вдовы наоборот оплачивают хорошо. Человек с доходами ниже 50 тыс рублей или выше 300 тыс. рублей лучше справиться с уплатой по счетам, чем те, кто зарабатывает иную сумму.

Итого: идеальный клиент - это вдовец с низкой зарплатой без детей, которому нужен кредит на ремонт жилья